# data preprocessing and cleaning ( IIT placement data)

In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [62]:
df_iit = pd.read_csv("/Users/anuragchaubey/smart-college-recommender/data/iit_placement.csv")
df_iit.head(5)

,Year,IIT Name,Overall Avg CTC (LPA),CSE Avg (LPA),ECE Avg (LPA),EE Avg (LPA),ME Avg (LPA),Civil Avg (LPA),Chemical Avg (LPA)
0,2020.0,IIT Bombay,20.08,20.08,17.91,20.08,20.65,Not Available,Not Available
1,2020.0,IIT Madras,18.12,25,18.12,27.28,18.12,11.51,Not Available
2,2020.0,IIT Kanpur,11.02,11.02,11.02,27.60,7.50,7.5,Not Available
3,2020.0,IIT Kharagpur,19.00,29.21,20,19.50,16.00,14.1,Not Available
4,2020.0,IIT Roorkee,16.00,18,15.5,13.12,10.33,Not Available,Not Available


In [63]:
# replace column names
df_iit.rename(columns={
    'IIT Name': 'institute_name',
    'Year' : 'year',
    'Overall Avg CTC (LPA)': 'overall_avg_ctc',
    'CSE Avg (LPA)': 'cse_avg_ctc',
    'ECE Avg (LPA)': 'ece_avg_ctc',
    'EE Avg (LPA)': 'ee_avg_ctc',
    'ME Avg (LPA)': 'me_avg_ctc',
    'Civil Avg (LPA)': 'civil_avg_ctc',
    'Chemical Avg (LPA)': 'chemical_avg_ctc'
}, inplace=True)

In [64]:
# replace all 'Not Available' strings with np.nan
df_iit.replace("Not Available", np.nan, inplace=True)

/var/folders/wz/4tdnltpd0j50hb4mv_kk_0140000gn/T/ipykernel_3536/1276220952.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_iit.replace("Not Available", np.nan, inplace=True)


In [65]:
# trim whitespaces from institute_name
df_iit['institute_name'] = df_iit['institute_name'].str.strip()

In [66]:
df_iit.isnull().sum()

year                  4
institute_name        4
overall_avg_ctc       6
cse_avg_ctc           6
ece_avg_ctc           7
ee_avg_ctc            8
me_avg_ctc            4
civil_avg_ctc       107
chemical_avg_ctc    119
dtype: int64

In [72]:
df_iit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              115 non-null    Int64  
 1   institute_name    115 non-null    object 
 2   overall_avg_ctc   113 non-null    float64
 3   cse_avg_ctc       110 non-null    float64
 4   ece_avg_ctc       111 non-null    float64
 5   ee_avg_ctc        111 non-null    float64
 6   me_avg_ctc        115 non-null    float64
 7   civil_avg_ctc     12 non-null     float64
 8   chemical_avg_ctc  0 non-null      float64
dtypes: Int64(1), float64(7), object(1)
memory usage: 8.6+ KB


In [71]:
# convert all columns to numeric except year and institute_name
cols_to_convert = df_iit.columns.difference(['Year', 'institute_name'])
df_iit[cols_to_convert] = df_iit[cols_to_convert].apply(pd.to_numeric, errors='coerce')

In [69]:
# convert year to int data type
df_iit['year'] = df_iit['year'].astype('Int64')

In [73]:
# remove fully blank rows
df_iit.dropna(how='all', inplace=True)

In [74]:
# check missing values
missing_values = df_iit.isnull().sum()
print(missing_values[missing_values > 0])

overall_avg_ctc       2
cse_avg_ctc           5
ece_avg_ctc           4
ee_avg_ctc            4
civil_avg_ctc       103
chemical_avg_ctc    115
dtype: int64


In [70]:
df_iit.head()

,year,institute_name,overall_avg_ctc,cse_avg_ctc,ece_avg_ctc,ee_avg_ctc,me_avg_ctc,civil_avg_ctc,chemical_avg_ctc
0,2020,IIT Bombay,20.08,20.08,17.91,20.08,20.65,NaN,NaN
1,2020,IIT Madras,18.12,25.00,18.12,27.28,18.12,11.51,NaN
2,2020,IIT Kanpur,11.02,11.02,11.02,27.60,7.50,7.50,NaN
3,2020,IIT Kharagpur,19.00,29.21,20.00,19.50,16.00,14.10,NaN
4,2020,IIT Roorkee,16.00,18.00,15.50,13.12,10.33,NaN,NaN


# handle missing values 

In [76]:
# fill missing values in overall_avg_ctc

# fill with data from forward or backward
df_iit['overall_avg_ctc'] = (
    df_iit.groupby('institute_name')['overall_avg_ctc']
          .transform(lambda x: x.ffill().bfill())
)

# if still left fill with overall mean
mean_overall = df_iit['overall_avg_ctc'].mean()
df_iit['overall_avg_ctc'] = df_iit['overall_avg_ctc'].fillna(mean_overall)


In [77]:
df_iit['overall_avg_ctc'].isnull().sum()

np.int64(0)

In [78]:
# filling missing values (cse_avg_ctc)

# fill with other cse data from same college
df_iit['cse_avg_ctc'] = (
    df_iit.groupby('institute_name')['cse_avg_ctc']
          .transform(lambda x: x.ffill().bfill())
)

# fill with correlation method (remaining)
cse_ratio = (df_iit['cse_avg_ctc'] / df_iit['overall_avg_ctc']).mean()
print("Average CSE to Overall ratio:", cse_ratio)

df_iit['cse_avg_ctc'] = df_iit['cse_avg_ctc'].fillna(
    df_iit['overall_avg_ctc'] * cse_ratio
)

Average CSE to Overall ratio: 1.2703877496291083


In [79]:
df_iit['cse_avg_ctc'].isnull().sum()

np.int64(0)

In [80]:
# filling missing values  (ece_avg_ctc)

# forward , backward filling
df_iit['ece_avg_ctc'] = (
    df_iit.groupby('institute_name')['ece_avg_ctc']
          .transform(lambda x: x.ffill().bfill())
)

# fill remaining values. ( using correlation method)
ece_ratio = (df_iit['ece_avg_ctc'] / df_iit['overall_avg_ctc']).mean()
print("Avg ECE to Overall Ratio:", ece_ratio)

df_iit['ece_avg_ctc'] = df_iit['ece_avg_ctc'].fillna(
    df_iit['overall_avg_ctc'] * ece_ratio
)

Avg ECE to Overall Ratio: 1.037341911412858


In [81]:
df_iit['ece_avg_ctc'].isnull().sum()

np.int64(0)

In [82]:
# filling missing values  (ee_avg_ctc)

# forward , backward filling
df_iit['ee_avg_ctc'] = (
    df_iit.groupby('institute_name')['ee_avg_ctc']
          .transform(lambda x: x.ffill().bfill())
)

# fill remaining values (correlation method)
ee_ratio = (df_iit['ee_avg_ctc'] / df_iit['overall_avg_ctc']).mean()
print("EE to Overall Ratio:", ee_ratio)

df_iit['ee_avg_ctc'] = df_iit['ee_avg_ctc'].fillna(
    df_iit['overall_avg_ctc'] * ee_ratio
)

EE to Overall Ratio: 1.138351042431576


In [83]:
df_iit['ee_avg_ctc'].isnull().sum()

np.int64(0)

In [84]:
# drop columns civil_avg_ctc and chemical_aavg_ctc (high % missing values)

df_iit.drop(['civil_avg_ctc', 'chemical_avg_ctc'], axis=1, inplace=True)

In [85]:
# sort the data on the basis of year and institute_name
df_iit = df_iit.sort_values(by=['year', 'institute_name']).reset_index(drop=True)

In [86]:
df_iit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   year             115 non-null    Int64  
 1   institute_name   115 non-null    object 
 2   overall_avg_ctc  115 non-null    float64
 3   cse_avg_ctc      115 non-null    float64
 4   ece_avg_ctc      115 non-null    float64
 5   ee_avg_ctc       115 non-null    float64
 6   me_avg_ctc       115 non-null    float64
dtypes: Int64(1), float64(5), object(1)
memory usage: 6.5+ KB


In [87]:
df_iit.isnull().sum()

year               0
institute_name     0
overall_avg_ctc    0
cse_avg_ctc        0
ece_avg_ctc        0
ee_avg_ctc         0
me_avg_ctc         0
dtype: int64

In [88]:
df_iit.shape

(115, 7)

In [89]:
df_iit.sample(5)

,year,institute_name,overall_avg_ctc,cse_avg_ctc,ece_avg_ctc,ee_avg_ctc,me_avg_ctc
76,2023,IIT Goa,17.19,21.52,16.99,16.99,14.76
26,2021,IIT Delhi,17.60,17.81,17.60,17.60,17.60
68,2022,IIT Varanasi,21.89,21.89,27.50,26.57,21.89
35,2021,IIT Jodhpur,24.38,12.00,12.00,12.00,12.00
21,2020,IIT Tirupati,11.00,14.00,11.00,14.00,10.95


In [90]:
df_iit.to_csv('/Users/anuragchaubey/smart-college-recommender/data/cleaned/iit_placement_cleaned.csv', index=False)
